In [152]:
import numpy as np
import pandas as pd
import matplotlib as plt
import os

path = os.getcwd()
player_time = pd.read_csv(path + '/player_time.csv')
matches = pd.read_csv(path + '/match.csv')

#radiant are slots 0-4, dire are slots 128-132

In [153]:
#print(player_time.head())
#print(player_time.shape)

In [154]:

player_time['mean_gold_rad'] = player_time[['gold_t_0', 'gold_t_1', 'gold_t_2', 'gold_t_3', 'gold_t_4']].sum(axis=1)
player_time['mean_xp_rad'] = player_time[['xp_t_0', 'xp_t_1', 'xp_t_2', 'xp_t_3', 'xp_t_4']].sum(axis=1)
player_time['mean_lh_rad'] = player_time[['lh_t_0', 'lh_t_1', 'lh_t_2', 'lh_t_3', 'lh_t_4']].sum(axis=1)
player_time['mean_gold_dire'] = player_time[['gold_t_128', 'gold_t_129', 'gold_t_130', 'gold_t_131', 'gold_t_132']].sum(axis=1)
player_time['mean_xp_dire'] = player_time[['xp_t_128', 'xp_t_129', 'xp_t_130', 'xp_t_131', 'xp_t_132']].sum(axis=1)
player_time['mean_lh_dire'] = player_time[['lh_t_128', 'lh_t_129', 'lh_t_130', 'lh_t_131', 'lh_t_132']].sum(axis=1)

player_time = player_time.drop(['gold_t_0', 'gold_t_1', 'gold_t_2', 'gold_t_3', 'gold_t_4', 'xp_t_0', 'xp_t_1', 'xp_t_2', 'xp_t_3', 'xp_t_4'], axis=1)
player_time = player_time.drop(['gold_t_128', 'gold_t_129', 'gold_t_130', 'gold_t_131', 'gold_t_132', 'xp_t_128', 'xp_t_129', 'xp_t_130', 'xp_t_131', 'xp_t_132'], axis=1)
player_time = player_time.drop(['lh_t_0', 'lh_t_1', 'lh_t_2', 'lh_t_3', 'lh_t_4', 'lh_t_128', 'lh_t_129', 'lh_t_130', 'lh_t_131', 'lh_t_132'], axis=1)

In [155]:
#print(player_time.head(10))

In [156]:
time_list = player_time['times']/60
player_time['times_minutes'] = time_list.astype(int)
#print(player_time.columns)
player_time = player_time[['match_id', 'times', 'times_minutes', 'mean_gold_rad', 'mean_xp_rad', 'mean_lh_rad', 'mean_gold_dire', 'mean_xp_dire', 'mean_lh_dire']]

In [157]:
print(player_time.head())
print(player_time.shape)

   match_id  times  times_minutes  mean_gold_rad  mean_xp_rad  mean_lh_rad  \
0         0      0              0              0            0            0   
1         0     60              1           1113          692            2   
2         0    120              2           2471         2376           10   
3         0    180              3           3490         3634           23   
4         0    240              4           4950         5285           33   

   mean_gold_dire  mean_xp_dire  mean_lh_dire  
0               0             0             0  
1            1370           692            10  
2            2726          2318            26  
3            4057          3543            40  
4            5500          5007            53  
(2209778, 9)


In [158]:
matches = matches.drop(['start_time', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_dire', 'barracks_status_radiant', 'first_blood_time', 'game_mode', 'negative_votes', 'positive_votes', 'cluster'], axis=1)
matches.head()

,match_id,duration,radiant_win
0,0,2375,True
1,1,2582,False
2,2,2716,False
3,3,3085,False
4,4,1887,True


In [166]:
player_time_final = player_time.merge(matches[['match_id', 'radiant_win']], on='match_id', how='left')

player_time_final['mean_gold_w'] = player_time_final.apply(
    lambda row: row['mean_gold_rad'] if row['radiant_win'] else row['mean_gold_dire'], axis=1
)

player_time_final['mean_gold_l'] = player_time_final.apply(
    lambda row: row['mean_gold_dire'] if row['radiant_win'] else row['mean_gold_rad'], axis=1
)

player_time_final['mean_xp_w'] = player_time_final.apply(
    lambda row: row['mean_xp_rad'] if row['radiant_win'] else row['mean_xp_dire'], axis=1
)

player_time_final['mean_xp_l'] = player_time_final.apply(
    lambda row: row['mean_xp_dire'] if row['radiant_win'] else row['mean_xp_rad'], axis=1
)

player_time_final['mean_lh_w'] = player_time_final.apply(
    lambda row: row['mean_lh_rad'] if row['radiant_win'] else row['mean_lh_dire'], axis=1
)

player_time_final['mean_lh_l'] = player_time_final.apply(
    lambda row: row['mean_lh_dire'] if row['radiant_win'] else row['mean_lh_rad'], axis=1
)

In [168]:
player_time_final.head()

,match_id,times,times_minutes,mean_gold_rad,mean_xp_rad,mean_lh_rad,mean_gold_dire,mean_xp_dire,mean_lh_dire,radiant_win,mean_gold_w,mean_gold_l,mean_xp_w,mean_xp_l,mean_lh_w,mean_lh_l
0,0,0,0,0,0,0,0,0,0,True,0,0,0,0,0,0
1,0,60,1,1113,692,2,1370,692,10,True,1113,1370,692,692,2,10
2,0,120,2,2471,2376,10,2726,2318,26,True,2471,2726,2376,2318,10,26
3,0,180,3,3490,3634,23,4057,3543,40,True,3490,4057,3634,3543,23,40
4,0,240,4,4950,5285,33,5500,5007,53,True,4950,5500,5285,5007,33,53


In [167]:
player_time_final.to_csv('player_time_means.csv')